# Data process

In [ ]:
import numpy as np
import pandas as pd
import zipfile
import datetime
import sys
import os

In [ ]:
data_path = r"D:\Dataset\FreddieMac\zipped_data"
unzipped_data_path = r"D:\Dataset\FreddieMac\unzipped_data"

## Unzip File (if not done yet)

In [ ]:
for file in os.listdir(data_path):
    if file.endswith(".zip"):
        file_name = os.path.join(data_path, file)
        with zipfile.ZipFile(file_name, "r") as zip_ref:
            zip_ref.extractall(unzipped_data_path)        

## Time Selection

In [ ]:
timestamp = ['Q42009','Q12010','Q22010','Q32010','Q42010','Q12011','Q22011','Q32011','Q42011']

## HPI data process

In [ ]:
HPI_msa_file = pd.read_csv('D:/Dataset/FreddieMac/HPI_AT_metro.csv',usecols=[1,2,3,4],dtype={'LOCATION':str})
HPI_state_file = pd.read_csv('D:/Dataset/FreddieMac/HPI_AT_state.csv')
HPI_file = pd.concat([HPI_msa_file, HPI_state_file],axis=0)

In [ ]:
available_msa = HPI_msa_file.LOCATION.unique()
available_msa_state = HPI_file.LOCATION.unique()

In [ ]:
HPI_file = HPI_file[HPI_file.YEAR >= 2004]
HPI_file['QUARTER_DATE'] = HPI_file.YEAR.astype(str) + 'Q' + HPI_file.QUARTER.astype(str)
HPI_file.drop(['YEAR','QUARTER'],axis=1,inplace=True)
HPI_file.set_index(['QUARTER_DATE','LOCATION'],inplace=True)
HPI_file.HPI.replace({0:np.nan},inplace=True)

In [ ]:
HPI_file.to_csv("D:\Dataset\FreddieMac\intermediate_data\HPI_data.csv")

## Orig Data process

In [ ]:
class OrigDataProcessor(object):
    
    def __init__(self, data):
        self.data = data.copy()
        self.delete = pd.Series(False, index=data.index)
        self.clean_funcs = [x for x in dir(OrigDataProcessor) if x.startswith('_clean')]
        
    def clean_table(self):
        for func in self.clean_funcs:
            eval('self.{}()'.format(func))
        print("{:.1%} rows are deleted".format(self.delete.sum()/len(self.data)))
        self.data = self.data[~self.delete]
        
    def _clean_01_fico(self):
        self.delete = self.delete | self.data.FICO.isnull()
        
    def _clean_02_first_time_home_buyer_flag(self):
        self.data.FIRST_TIME_HOME_BUYER_FLAG.fillna('U', inplace=True)
        
    def _clean_03_msa(self):
        unavailable_msa_mask = ~self.data.MSA.isin(available_msa)
        self.data.loc[unavailable_msa_mask, 'MSA'] = self.data.loc[unavailable_msa_mask, 'PROP_STATE']
        self.delete = self.delete | (~self.data.MSA.isin(available_msa_state))
        
    def _clean_04_mortgage_insurance_pct(self):
        self.delete = self.delete | self.data.MORTGAGE_INSURANCE_PCT.isnull()
        
    def _clean_05_num_of_units(self):
        self.delete = self.delete | self.data.NUM_OF_UNITS.isnull()
        
    def _clean_06_occupancy_status(self):
        self.delete = self.delete | self.data.OCCUPANCY_STATUS.isnull()
        
    def _clean_07_orgn_cltv(self):
        self.delete = self.delete | self.data.ORGN_CLTV.isnull()
        
    def _clean_08_loan_size(self):
        self.delete = self.delete | self.data.LOAN_SIZE.isnull()
        
    def _clean_09_orgn_ltv(self):
        self.delete = self.delete | self.data.ORGN_LTV.isnull()
        self.delete = self.delete | (self.data.ORGN_LTV > self.data.ORGN_CLTV)
        
    def _clean_10_orgn_rate(self):
        self.delete = self.delete | self.data.ORGN_RATE.isnull()
        
    def _clean_11_channel(self):
        self.delete = self.delete | self.data.CHANNEL.isnull()
        
    def _clean_12_prop_type(self):
        self.delete = self.delete | self.data.PROP_TYPE.isnull()
        
    def _clean_13_loan_id(self):
        self.delete = self.delete | self.data.LOAN_ID.isnull()
        
    def _clean_14_loan_purpose(self):
        self.delete = self.delete | self.data.LOAN_PURPOSE.isnull()
        
    def _clean_15_num_of_borrowers(self):
        self.delete = self.delete | self.data.NUM_OF_BORROWERS.isnull()
        
    def _clean_16_SUPER_CONFORMING_FLAG(self):
        self.data.SUPER_CONFORMING_FLAG.fillna('N', inplace=True)

In [ ]:
orig_usecols=[1,3,5,6,7,8,9,11,12,13,14,17,18,20,21,23,26]
orig_cols = ['FICO',
             'FIRST_TIME_HOME_BUYER_FLAG',
             'MSA',
             'MORTGAGE_INSURANCE_PCT',
             'NUM_OF_UNITS',
             'OCCUPANCY_STATUS',
             'ORGN_CLTV',
             'LOAN_SIZE',
             'ORGN_LTV',
             'ORGN_RATE',
             'CHANNEL',
             'PROP_STATE',
             'PROP_TYPE',
             'LOAN_ID',
             'LOAN_PURPOSE',
             'NUM_OF_BORROWERS',
             'SUPER_CONFORMING_FLAG']

In [ ]:
processed_orig_file_list = []

orig_file_names = ['historical_data1_{}.txt'.format(x) for x in timestamp]

for file_name in orig_file_names:
    orig_data_path = os.path.join(unzipped_data_path, file_name)
    orig_data = pd.read_csv(orig_data_path,
                            header=None,
                            names=range(1,27),
                            sep='|',
                            dtype={1:float, 5:str, 6: float},
                            na_values='   ')[orig_usecols]
    orig_data.columns = orig_cols
    processor = OrigDataProcessor(orig_data)
    processor.clean_table()
    processed_orig_file_list.append(processor.data)

In [ ]:
orig_data_merged = pd.concat(processed_orig_file_list, axis=0)
orig_data_merged.set_index('LOAN_ID', inplace=True)

In [ ]:
orig_data_merged.to_csv("D:\Dataset\FreddieMac\intermediate_data\orig_data.csv")
del(processed_orig_file_list)

## Time data process

In [ ]:
time_usecols=np.array([1,2,4,5,9])-1

time_cols = ['LOAN_ID',
             'DATE',
             'DELQ_OBS',
             'LOAN_AGE',
             'ZERO_BALANCE_CODE']

In [ ]:
class TimeDataProcessor(object):
    
    def __init__(self, data, loanID):
        self.data = data.copy()
        self.delete = pd.Series(False, index=data.index)
        self.clean_funcs = [x for x in dir(TimeDataProcessor) if x.startswith('_clean')]
        self.loanID = loanID
        
    def clean_table(self):
        for func in self.clean_funcs:
            eval('self.{}()'.format(func))
        print("{:.1%} rows are deleted".format(self.delete.sum()/len(self.data)))
        self.data = self.data.loc[~self.delete, ['LOAN_ID','QUARTER_DATE','DELQ_OBS','ZERO_BALANCE_CODE']]
        self.data.set_index('LOAN_ID',inplace=True)

    def _clean_01_loan_id(self):
        self.delete = self.data['LOAN_ID'].isnull() | (~self.data['LOAN_ID'].isin(self.loanID))
        
    def _clean_02_date(self):
        year, month = divmod(self.data['DATE'], 100)
        quarter = (month-1) // 3 + 1
        self.data['QUARTER_DATE'] = year.astype(str) + 'Q' + quarter.astype(str)
    
    def _clean_03_loan_age(self):
        self.delete = self.delete | (self.data['LOAN_AGE'] > 60) | self.data['LOAN_AGE'].isnull()
        
    def _clean_04_delq_obs(self):
        self.data['DELQ_OBS'].replace({'XX':np.nan, 'R':'99'}, inplace=True)
        self.data['DELQ_OBS'] = self.data['DELQ_OBS'].astype(float)

In [ ]:
processed_time_file_list = []

time_file_names = ['historical_data1_time_{}.txt'.format(x) for x in timestamp]

for file_name in time_file_names:
    time_data_path = os.path.join(unzipped_data_path, file_name)
    time_data = pd.read_csv(time_data_path,
                            header=None,
                            usecols=time_usecols,
                            names=time_cols,
                            sep='|',
                            dtype={'DELQ_OBS':str, 'DATE': int, 'ZERO_BALANCE_CODE':str},
                            na_values=' ')
    processor = TimeDataProcessor(time_data,orig_data_merged.index.values)
    processor.clean_table()
    processor.data = processor.data.join(orig_data_merged[['MSA']],how='left')
    processor.data = processor.data.merge(HPI_file, left_on=['QUARTER_DATE','MSA'], right_index=True, how='left')
    processed_time_file_list.append(processor.data)

In [ ]:
time_data_merged = pd.concat(processed_time_file_list, axis=0)
del(processed_time_file_list)

In [ ]:
grp = time_data_merged.groupby(time_data_merged.index)
hpi_min_max = grp.agg({'HPI':{'min','max'}})['HPI']
hpi_orig = grp.first().HPI.to_frame('orig')
hpi_info = hpi_orig.join(hpi_min_max, how='left')

In [ ]:
hpi_info.columns=['HPI_ORIG','HPI_MIN','HPI_MAX']
hpi_info.to_csv("D:\Dataset\FreddieMac\intermediate_data\loan_hpi_info.csv")

In [ ]:
default_1 = time_data_merged[(time_data_merged.DELQ_OBS >=5)|(time_data_merged.ZERO_BALANCE_CODE.isin(['03','09']))].index.unique()
default_2 = time_data_merged[(time_data_merged.DELQ_OBS >=5)|(time_data_merged.ZERO_BALANCE_CODE.isin(['03','06','09']))].index.unique()

## Data Merging

In [ ]:
orig_data_merged = orig_data_merged.join(hpi_info,how='left')

In [ ]:
orig_data_merged = orig_data_merged[orig_data_merged.HPI_ORIG.notnull()]

In [ ]:
orig_data_merged.loc[:,'IND_DEFAULT_1'] = 0
orig_data_merged.loc[orig_data_merged.index.isin(default_1), 'IND_DEFAULT_1'] = 1
orig_data_merged.loc[:,'IND_DEFAULT_2'] = 0
orig_data_merged.loc[orig_data_merged.index.isin(default_2), 'IND_DEFAULT_2'] = 1

In [ ]:
orig_data_merged.to_csv("D:\Dataset\FreddieMac\intermediate_data\integrated_data.csv")